In [1]:
import torch
import numpy as np
from transformers import AutoTokenizer, Trainer
from datasets import Dataset

/home/jonhue/projects/jonhue/twitter-sentiment-classification/venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from loading import load_test
from preprocessing import preprocess
from submission import prepare_submission
from bert import tokenize, get_BERT

[nltk_data] Downloading package stopwords to /home/jonhue/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jonhue/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [4]:
MODEL = 'test_large_bs'
TOKENIZER = 'bert-base-uncased'
PREPROCESSING = None

In [5]:
df_test = load_test(x_col='text')
preprocess(df_test, flags=PREPROCESSING, x_col='text')
dataset_test = Dataset.from_pandas(df_test)

In [6]:
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER)
test_tokenized = tokenize(dataset_test, tokenizer, path=f'bert/cache/test_tokenized__{TOKENIZER}')

Parameter 'function'=<function tokenize.<locals>.tokenize_function at 0x7f9176315e50> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
100%|██████████| 10/10 [00:01<00:00,  7.92ba/s]


In [7]:
model = get_BERT(MODEL, device)

In [ ]:
trainer = Trainer(model, tokenizer=tokenizer)

In [ ]:
test_pred = trainer.predict(test_tokenized)
test_pred = np.argmax(test_pred.predictions, axis=1)
test_pred[test_pred == 0] = -1

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: __index_level_0__, text, token_type_ids. If __index_level_0__, text, token_type_ids are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 10000
  Batch size = 8


In [ ]:
prepare_submission(test_pred)